# 字词嵌入

### 字节对编码

FastText 模型中 提取的所有字词都必须是指定的长度 例如 3 or 6 因此词表大小不能预定义  
为了在固大小的词表中 允许可变长度的字词 我们可以使用 **字节对编码 BPE** 压缩算法来提取字词

In [34]:
import collections

Symbols = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '_', '[UNK]']

因为我们不考虑跨越词边界的符号对 所以我们只需要一个词典 raw_token_freqs 讲词映射到数据集中的频率 出现次数  
注意：特殊符号 '_' 被附加到每个词的尾部  
由于 我们仅从单个字符和特殊符号的词开始合并处理
因此每个词 ‘词典 token_freqs的键’ 内的每对连续字符之间插入空格  
空格是词中符号之间的分隔符

In [35]:
raw_tokens_freqs = {'fast_': 4, 'faster_': 3, 'tall_': 5, 'taller_': 4}
token_freqs = {}
for token, freq in raw_tokens_freqs.items():
    token_freqs[' '.join(list(token))] = raw_tokens_freqs[token]
token_freqs

{'f a s t _': 4, 'f a s t e r _': 3, 't a l l _': 5, 't a l l e r _': 4}

我们定义以下 get_max_freq_pair 函数，其返回词内出现最频繁的连续符号对，其中词来自输入词典 token_freqs 的健

In [36]:
def get_max_freq_pair(token_freqs):
    pairs = collections.defaultdict(int) # 当 pair 不存在时，返回 0
    for token, freq in token_freqs.items():
        symbols = token.split() # 将 token 拆分为单个字符
        for i in range(len(symbols) - 1):
            pairs[symbols[i], symbols[i + 1]] += freq # 统计 pair 出现的次数
    return max(pairs, key=pairs.get) # 返回频率最大的 pair

In [37]:
def merge_symbols(max_freq_pair, token_freqs, symbols):
    symbols.append(''.join(max_freq_pair)) # 将 pair 合并为一个新的字符
    new_token_freqs = {} # 存储新的 token 频率
    for token, freq in token_freqs.items(): # 更新 token 频率
        new_token = token.replace(' '.join(max_freq_pair), ''.join(max_freq_pair)) # 将 token 中的 pair 替换为新字符
        new_token_freqs[new_token] = token_freqs[token] # 更新 token 频率
    return new_token_freqs

In [38]:
num_epochs = 10
for i in range(num_epochs):
    max_freq_pair = get_max_freq_pair(token_freqs)
    token_freqs = merge_symbols(max_freq_pair, token_freqs, Symbols)
    print(f'合并#{i + 1}:', max_freq_pair)

合并#1: ('t', 'a')
合并#2: ('ta', 'l')
合并#3: ('tal', 'l')
合并#4: ('f', 'a')
合并#5: ('fa', 's')
合并#6: ('fas', 't')
合并#7: ('e', 'r')
合并#8: ('er', '_')
合并#9: ('tall', '_')
合并#10: ('fast', '_')


In [39]:
print(Symbols)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '_', '[UNK]', 'ta', 'tal', 'tall', 'fa', 'fas', 'fast', 'er', 'er_', 'tall_', 'fast_']


In [40]:
print(list(token_freqs.values()))

[4, 3, 5, 4]


注意 字节对编码的结果取决于正在使用的数据集  
我们还可以使用从一个数据集中学习到的子词来切分另一个数据集的词  

In [41]:
def segment_BPE(tokens, symbols):
    outputs = []
    for token in tokens:
        start, end = 0, len(token)
        cur_output = [] # 存储分词结果
        while start < len(token):
            if token[start:end] in symbols:
                cur_output.append(token[start:end])
                start = end
                end = len(token)
            else:
                end -= 1
        if start < len(token):
            cur_output.append('[UNK]')
        outputs.append(' '.join(cur_output))
    return outputs

In [42]:
tokens = ['tallest_', 'fatter_']
print(segment_BPE(tokens, Symbols))

['tall e s t _', 'fa t t er_']
